#**SQLAlchemy**
Diversas aplicaçōes escritas em Python precisam armazenar dados relacionais, usando bancos de dados como PostgreSQL, MySQL, Oracle e outros.

O **SQLAlchemy** é um framework de mapeamento objeto-relacional SQL (ORM) de código aberto. 

Mapeamento objeto-relacional ou simplesmente **ORM** é uma técnica de programação que auxilia na conversão de dados entre banco de dados relacionais e linguagens de programação que são orientadas à objetos.

A utilização deste Framework faz com que o desenvolvedor reduza a programação de acesso ao banco de dados, desta forma, obtendo uma produtividade significativa no desenvolvimento de suas aplicações. Ele realiza todas as conversões necessárias entre o modelo relacional e o modelo orientado a objetos de maneira automática, geralmente da seguinte forma:
- Cada classe é interpretada com uma tabela;
- Cada linha de uma determinada tabela, junto com seu relacionamento é tratada como instância do objeto relacionado à tabela em questão.
- Sua utilização também visa retirar a necessidade do desenvolvedor de software em se preocupar com a linguagem SQL (geração dos comandos SQL) e com as conversões necessárias entre os diferentes tipos de dados, já que tudo isto fica a cargo do Framework ORM.

Conseguimos utilizar banco de dados relacionais por meio de objetos escritos em Python, não se preocupando em como este BD está funcionando (relações, campos, etc), apenas mapeando sua estrutura.

##Instalação
Para utilizar a biblioteca SQLAlchemy, é preciso primeiramente executar o comando abaixo em uma janela de comandos de seu sistema operacional:
```python
pip install sqlalchemy
```


##Importação
A primeira coisa que o programa python deve fazer é importar a biblioteca SQLAlchemy para o programa.

```python
import sqlalchemy
```

##Conexão
Agora vamos conectar o programa a um banco de dados.

Neste exemplo, utilizaremos um banco de dados SQLite, mas é possível o uso de outros bancos, mudando apenas a string de conexão que é passada como parâmetro.

```python
engine = sqlalchemy.create_engine('sqlite:///infinity.db',echo=True)
```

>OBS: O parâmetro "echo=True" é opcional. Quando utilizado, exibe todas os comandos SQL que a biblioteca utiliza quando aciona a engine.
>>IMPORTANTE: Atanção ao indicar a localização do banco de dados. Lembre-se que o comportamento padrão do SQLite é criar um novo banco caso a string de conexão apote para um banco que não exista.

Veja exemplos de strings de conexão para outros bancos de dados neste link: https://docs.sqlalchemy.org/en/14/core/engines.html


##Criando o Banco de Dados
Para criar um banco de dados ou conectar-se a um já existente, é preciso, primeiramente, definir uma ***engine*** de acesso e declarar uma base de dados que realiza o mapeamento objeto-relacional (ORM).

```python
# engine
engine = sqlalchemy.create_engine('sqlite:///contatos.db',echo=True)

# mapeamento ORM
from sqlalchemy.orm import declarative_base
Base = declarative_base()
```

##Criando Tabelas
Antes de criar as tabelas, é preciso importar os objetos e tipos de dados definidos pelo ***SQLAlchemy*** para associá-los a cada coluna das tabelas.

Exemplo:
```python
from sqlalchemy import Column, String, Integer
```
Agora é só criar as classes. **Cada classe criada vai ser associada a uma tabela no banco de dados** pelo SQLAlchemy.
```python
class Pessoa(Base):
    __tablename__ = 'tab_pessoas'
    idPessoa = Column(Integer, primary_key=True)
    nomePessoa = Column(String(20))
    idadePessoa = Column(Integer)```
```

Por fim, as classes(tabelas) definidas são incorporadas à engine do banco.

```python
Base.metadata.create_all(engine)
```

##Incluindo Dados em uma Tabela
Como as tabelas são associadas às classes, para inserir um registro em uma tabela, basta **instanciar uma classe**, criar uma **sessão** com o banco de dados e acionar o método **"add"**, seguido de um **"commit"**.

```python
# instanciando a classe
novaPessoa = Pessoa(idPessoa=1,nomePessoa='Pedro',idadePessoa=25)

# criando uma sessão no banco de dados
from sqlalchemy.orm import sessionmaker
SessaoBD = sessionmaker(bind=engineBD) # cria uma classe "Sessao"

sessao = SessaoBD() # intancia um objeto do tipo "SessaoBD" chamado "sessao"

# inserindo dados de contatos
sessao.add(novaPessoa)
sessao.commit() # confirma adição de dados no banco
# sessao.rollback() # desfaz adição de dados no banco
```
O método **"add"** insere apenas um registro. Para inserir vários registros de um vez, usa-se uma lista e o método **"add_all(lista)"**.
```python
lista_pessoas = [pessoa1,pessoa2,pessoa3]
sessao.add_all(lista_pessoas)
```




##Consultando Dados
Há diversas maneiras de consultar dados.
Todas elas passam pela criação de uma **sessão** com o banco de dados e a execução de um método **"query"**.
A depender da forma como o método é acionado, a resposta do método pode ser um objeto, uma lista de objetos ou uma tupla.

```python
# consultando pessoas --------------------------------
resposta = sessao.query(Pessoa) # retorna uma lista de pessoas
for resp in resposta:
    print('Consulta 1 =',resp.id, resp.nome)

# consultando pessoas --------------------------------
resposta = sessao.query(Pessoa).filter_by(nome='Ana') # retorna uma lista de pessoas
for resp in resposta:
    print('Consulta 2 =',resp.nome)

# consultando pessoas --------------------------------
resposta = sessao.query(Pessoa).filter_by(nome='Paulo').first() # retorna apenas 1 pessoa
print('Consulta 3 =',resposta.nome)

# consultando pessoas --------------------------------
for resp in sessao.query(Pessoa).order_by(Pessoa.nome): # retorna uma lista ordenada de pessoas
    print('Consulta 4 =',resp.id, resp.nome)

# consultando pessoas --------------------------------
for info in sessao.query(Pessoa.nome,Pessoa.id): # retorna uma lista de tuplas (nome,id)
    print('Consulta 5 =', info)

# consultando pessoas --------------------------------
print('Consulta 6 =',s1.query(Pessoa).filter(Pessoa.nome=='Marcos').first().nome) # retorna 1 única pessoa

# consultando pessoas --------------------------------
print('Consulta 7 =',s1.query(Pessoa).count()) # retorna a quantidade de pessoas

```

##Alterando Dados em uma Tabela
Lembrando: As tabelas são associadas às classes.

Um objeto recuperado de uma query ou já instanciado (e criado no banco) só precisa alterar os dados no objeto e disparar um novo **commit()**.
```python
# alterando dados
pessoa = sessao.query(Pessoa).filter(Pessoa.nome=='Marcos').first()
if pessoa != None:
  pessoa.nome = 'Alberto'
  sessao.commit()
  print('Apaguei uma pessoa')
```

##Excluindo Dados de uma Tabela
Um objeto recuperado de uma query ou já instanciado (e criado no banco) é removido da tabela usando o método **"delete(objeto)"** e disparando um **commit()**.
```python
# excluido dados
pessoa = sessao.query(Pessoa).filter(Pessoa.nome=='Ana').first()
if pessoa != None:
  sessao.delete(pessoa)
  sessao.commit()
  print('Apaguei uma pessoa')
```

##Dados Dependentes de Commit() ou Rollback()
Para verificar se a sessão corrente tem dados inseridos, alterados ou excluidos que ainda necessitem de confirmação no banco de dados, basta verificar os objetos **"new"** (para dados novos) e **"dirty"** (para dados já existentes, mas alterados, ou deletados).
```python
print(sessao.new)
print(sessao.dirty)
```